In [46]:
from collections import Counter, defaultdict
import re
import numpy as np
import re
import os

Вспомним, как писать бесконечные генераторы с помощью ```yield```

In [2]:
def num_generator():
    i= 0
    while True:
        yield i
        i += 1     

In [3]:
gen = num_generator()
print(next(gen))
print(next(gen))
print(next(gen))

0
1
2


А если хочется получить сразу несколько первых элементов из генератора?

**Задание** Напишите функцию-генератор, которая будет выдавать элементы до тех пор, пока не наберется нужное количество (`n`).

In [4]:
def take(generator, n):
    try:
        for i in range(n):
            yield (next(generator))
        
    except StopIteration:
        pass
    
list(take(num_generator(), 5))

[0, 1, 2, 3, 4]

**Задание** Напишите генератор, который будет выдавать только четные числа.

In [5]:
def even_num_generator(num_generator):
    while True:
        num = next(num_generator)
        if num % 2 == 0:
            yield num

list(take(even_num_generator(num_generator()), 5))

[0, 2, 4, 6, 8]

## N-граммная языковая модель

In [6]:
!head poetry.txt -n 5

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


***Задание*** Посчитайте частотности слов в файле за ```O(1)``` памяти, то есть без ```f.read()```

In [7]:
path = os.path.abspath("C://Users//Asus//ColingML//HW1//poetry.txt")

In [23]:
word_counts = Counter()
with open('poetry.txt', encoding='utf-8') as f:
    text = " ".join([l.lower().strip() for l in f])
    text = re.sub('[^\\w\\s]',' ', text) 
    words = [word for word in text.split()]
    word_counts.update(words)
print(word_counts.most_common(100))

[('и', 111567), ('в', 75623), ('не', 50963), ('я', 38550), ('на', 34623), ('как', 26493), ('с', 25614), ('что', 21918), ('ты', 18016), ('а', 16591), ('но', 15903), ('он', 14102), ('мне', 14093), ('за', 11356), ('то', 11194), ('все', 11147), ('к', 10751), ('по', 10471), ('о', 9791), ('мы', 9353), ('так', 8790), ('от', 8694), ('из', 8230), ('у', 7726), ('где', 7439), ('меня', 7081), ('когда', 6246), ('под', 6071), ('его', 5865), ('ни', 5818), ('нет', 5795), ('их', 5604), ('же', 5429), ('над', 5319), ('это', 5269), ('мой', 5182), ('кто', 5102), ('тебя', 5031), ('вот', 4929), ('да', 4727), ('только', 4712), ('она', 4693), ('без', 4652), ('до', 4568), ('там', 4502), ('бы', 4496), ('был', 4470), ('для', 4385), ('нам', 4073), ('нас', 3986), ('ли', 3889), ('всё', 3882), ('тебе', 3869), ('вы', 3743), ('во', 3714), ('ее', 3592), ('еще', 3575), ('сердце', 3547), ('чтоб', 3437), ('лишь', 3376), ('быть', 3216), ('жизнь', 3204), ('если', 3201), ('есть', 2963), ('они', 2881), ('может', 2760), ('было'

Для создания нашей N-граммной модели будем накапливать статистику для всех строк стихотворений. 
Генерировать стихотворения будем построчно. Для этого нужно научить генерировать "первое слово" и "последнее слово" строки.

**Задание** Напишите `parser` - функцию, которая будет выдавать поток токенов из файла. При этом каждая "строка" стихотворения должна начинаться с токена ```<BOS>``` и ```<EOS>```.
Добавьте предобработку: фильтруйте все строки длиной менее 20 символов. Удалите все знаки препинания. Приведите к нижнему регистру

In [8]:
def parser(path):
    with open(path, encoding="utf-8") as f:
        all_tokens = []
        for line in f:
            if len(line) >= 20:
                line = re.sub('[^\w\s]',' ', line.strip().lower())
                line = "<BOS> " + line + " <EOS>"
                words = line.split()
                for w in words:
                    yield w

In [9]:
parse = parser(path)  

Напишем N-граммную языковую модель по стишкам.

Языковая модель умеет оценивать вероятности $\mathbf{P}(w_1, \ldots, w_n) = \prod_k \mathbf{P}(w_k|w_{k-1}, \ldots, w_{1})$.

N-граммная языковая модель приближает эту вероятность, используя предположение, что вероятность токена зависит только от недавней истории: $\mathbf{P}(w_k|w_1, \ldots, w_{k-1}) = \mathbf{P}(w_k|w_{k-1}, \ldots, w_{k-N + 1})$.

Для начала нужно собрать статистику. Для простоты будем работать с биграмной моделью, а значит - нужно собрать информацию:

- о биграммных частотностях $(w_{i-1}) \to C(w_i)$
- о триграммных частотностях $(w_{i-2}, w_{i-1}) \to C(w_i)$

Также хочется сохранять в статистику n-gram информацию о начале и коцне стишка.

**Задание** Напишите функцию, которая будет из потока токенов формировать и выдавать наружу пары (ngram, next_word).

Каждый стишок должен начинаться с токена ```<BOS>``` и заканчиваться ```<EOS>``` (beginning of sequence, end of sequence).

In [10]:

def compose_ngram(tokens_stream):
    
    tokens = []
    while True:
        try:
            token = next(tokens_stream)
            tokens.append(token)
            if token == "<EOS>":
                tokens.append("<EOS>")
                pass
                
                try:
                    for i in range(len(tokens)):
                        yield (((tokens[i], tokens[i+1]), tokens[i+2]))
                        yield (((tokens[i], tokens[i+1], tokens[i+2]), tokens[i+3]))
                except IndexError:
                    pass
                        
                
                
                
                tokens = []
                
                
        except StopIteration:
            break
    
    
    
    
# Подсказки:
# Что сохраняем, если token --> <BOS>?
# Что делаем, если token --> <EOS>?

Соберем статистику:

In [11]:
ngrams_counter = defaultdict(Counter)
for ngram in compose_ngram(parse):
    ngrams_counter[tuple(ngram[0])][ngram[1]] += 1

Теперь генерировать будем так: есть стартовый токен. Проверяем последнюю триграмму в статистике. Если есть, генерируем с помощью нее новое слово. Если нет, ищем биграму (она должна быть).
Если засемплили ```<EOS>``` --> завершаем генерацию и строку.
Генерировать будем четверостишья (стихотворения из 4 строк)

In [192]:
def sample_token(ngrams_counter, ngram):
    probs = np.array(list(ngrams_counter[ngram].values()))
    probs = probs / np.sum(probs)
    return np.random.choice(list(ngrams_counter[ngram]), p=probs)  
  
def generate_line(ngrams_counter):
    
    buffer = ['<BOS>']
    
    while True:
        
        initial_token = [ngram for ngram in ngrams_counter.keys() if len(ngram) == 3]
        ind = np.random.choice(len(initial_token))
        initial_token = initial_token[ind]
        
        if initial_token[2] != "<EOS>":
            
        
            word = sample_token(ngrams_counter, initial_token)

            if  "<EOS>" not in word:
                buffer.append(word)
                
                
                
                new_token = (initial_token[1], initial_token[2], word)
                
                

                if new_token in ngrams_counter:
                    new_word = sample_token(ngrams_counter, new_token)
                    if new_word == "<EOS>":
                        break
                    buffer.append(new_word)
                    
                else:
                    pass

                #else:
                 #   new_token = (initial_token[2], word)
                  #  new_word = sample_token(ngrams_counter, new_token)
                   # if new_word == "<EOS>":
                    #    break
                    #buffer.append(new_word)
                    
            else:
                    pass
        else:
            pass
                    
    buffer.append("<EOS>")    
    return (buffer)
            
        
        
            
        
    
    

In [185]:
generate_line(ngrams_counter)

['<BOS>',
 'не',
 'желаю',
 'субботних',
 'приложений',
 'осеннего',
 'бала',
 'его',
 'играя',
 'перенесет',
 '<EOS>']

###### Если брать еще и биграммы, то получается какая-то ерунда, поэтому пока оставлю как есть. Постараюсь придумать что-то получше

In [195]:
for _ in range(4):
    print(' '.join(generate_line(ngrams_counter)).strip('<EOS> <BOS>'))

все крепчает но сколь заклятье
тины водорослей и шумел ладонь
легким скоком непрочен
её уже старою курою в серсо громко и хочешь ты бурьена
